In [1]:
import os
datapath = "/net/tscratch/people/plgjsikora/data"

In [2]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical

class SHLDataProviderRaw:
  def __init__(self, root_path:str, flag: str, mode: str, location: str = "Hips", sensor: str = "Acc_x", silent = True, four_locs = False):
    '''
    | Setup data provider for SHL 2025 Challenge
    | Args:
    | root_path: path to root data folder
    | flag: one of train, validation, test
    | mode: one of singlefile, singlefolder, all
    | location: where to load from (Hips/Bag etc)
    | sensor: which file to load (Acc_x/Mag_z etc)
    | silent: supress print statements
    | Example usage:
    | provider = SHLDataProviderRaw(datapath, "train", "singlefolder", location="Bag")
    | x, y = provider.load_data()
    '''
    self.mode = mode.lower()
    self.flag = flag.lower()
    self.location = location.capitalize()
    self.sensor = sensor.capitalize()
    self.silent = silent
    if self.flag not in ["test", "train", "validation"]:
      raise ValueError(f"Unexpected flag received: got {self.flag}")
    if self.mode not in ["singlefile", "singlefolder", "all"]:
      raise ValueError(f"Unexpected mode received: got {self.mode}")
    if self.flag == 'train' and four_locs:
      self.datapath = os.path.join(root_path, "train_4_locations")
    else:
      self.datapath = os.path.join(root_path, self.flag)

  def load_data(self):
    if self.mode == "singlefile":
      x_data, y_data = self._load_from_file()
    elif self.mode == "singlefolder":
      x_data, y_data = self._load_from_folder()
    elif self.mode == "all":
      x_data, y_data = self._load_all()
    return np.array(x_data), np.array(y_data)

  def _load_label_file(self, labelpath):
      if not self.silent:
        print(f"Loading labels from: {labelpath}")
      y_data = np.loadtxt(labelpath, dtype=int)
      y_data[np.isnan(y_data)] = 1
      y_data = np.median(y_data, axis=1).astype(int)
      y_data = y_data - 1
      y_data = to_categorical(y_data, len(np.unique(y_data)))
      return y_data

  def _load_location_data(self, path):
      def load_txt_csv(path):
            if not self.silent:
                print(f"Loading data from: {path}")
            if self.flag == 'test':
                np_data = np.loadtxt(path, dtype=np.float32, delimiter=",")
            else:
                df = pd.read_csv(path, header=None, delim_whitespace=True, engine='python')
                np_data = df.to_numpy()
            return np_data
      acc_x = load_txt_csv(f'{path}/Acc_x.txt')
      acc_y = load_txt_csv(f'{path}/Acc_y.txt')
      acc_z = load_txt_csv(f'{path}/Acc_z.txt')
      gyr_x = load_txt_csv(f'{path}/Gyr_x.txt')
      gyr_y = load_txt_csv(f'{path}/Gyr_y.txt')
      gyr_z = load_txt_csv(f'{path}/Gyr_z.txt')
      mag_x = load_txt_csv(f'{path}/Mag_x.txt')
      mag_y = load_txt_csv(f'{path}/Mag_y.txt')
      mag_z = load_txt_csv(f'{path}/Mag_z.txt')
      data = np.stack([acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z], axis=2)
      return data
  
  def _load_from_folder(self):
      '''
      Load all sensors from folder (location)
      '''
      if self.flag == "test":
          filepath = self.datapath
          data = self._load_location_data(filepath)
          return data
      else:
          filepath = os.path.join(self.datapath, self.location)
          labels = self._load_label_file(os.path.join(filepath, "Label.txt"))
          data = self._load_location_data(filepath)
          return data, labels
  def _load_all(self):
      if self.flag == 'test':
          data = self._load_from_folder()
          return data
      self.location = "Hips"
      hips_data, hips_labels = self._load_from_folder()
      self.location = "Torso"
      torso_data, torso_labels = self._load_from_folder()
      self.location = "Bag"
      bag_data, bag_labels = self._load_from_folder()
      self.location = "Hand"
      hand_data, hand_labels = self._load_from_folder()
      data = np.concatenate([hips_data, torso_data, bag_data, hand_data], axis = 0)
      labels = np.concatenate([hips_labels, torso_labels, bag_labels, hand_labels], axis=0)
      return data, labels


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from tensorflow.keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from tensorflow.keras import regularizers

class ModelProvider:
  def __init__(self, seq_len = 500, nfeatures = 9, nclasses = 8):
    '''
    | Initializer model provider
    | Args:
    | seq_len - length of time series analysed
    | nfeatures - number of features of each time point
    | nclasses - number of target classes
    | Example usage:
    | provider = ModelProvider(500, 9, 8)
    | model = provider.get_model()
    '''
    self.seq_len = seq_len
    self.nfeatures = nfeatures
    self.nclasses = nclasses
    self.model = None


  def build_model(self):
    '''
    | Build model for SHL Challenge
    | Reimplementation of a model from https://github.com/titu1994/MLSTM-FCN for newer keras
    '''

    def squeeze_excite_block(input):
      ''' Create a squeeze-excite block
      Args:
          input: input tensor
          filters: number of output filters
          k: width factor

      Returns: a keras tensor
      '''
      filters = input.shape[-1] # channel_axis = -1 for TF

      se = GlobalAveragePooling1D()(input)
      se = Reshape((1, filters))(se)
      se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
      se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
      se = multiply([input, se])
      return se

    ip = Input(shape=(self.seq_len, self.nfeatures), dtype=float)
    x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.4)(x)

    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform', data_format='channels_last', kernel_regularizer=regularizers.L2(1e-4))(ip)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)
    y = Dropout(0.4)(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform', data_format='channels_last', kernel_regularizer=regularizers.L2(1e-4))(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)
    y = Dropout(0.4)(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform', data_format='channels_last', kernel_regularizer=regularizers.L2(1e-4))(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = GlobalAveragePooling1D()(y)
    y = Dropout(0.4)(y)

    x = concatenate([x, y])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(self.nclasses, activation='softmax')(x)
    self.model = Model(ip, out)

  def from_weights(self, weightpath):
    '''
    Get model from existing weights
    '''
    self.build_model()
    self.model.load_weights(weightpath)
    return self.model

  def get_model(self):
    if self.model is None:
      self.build_model()
    return self.model

In [4]:
mprovider = ModelProvider(nfeatures = 9)
#model_path = "/net/tscratch/people/plgjsikora/models/MLSTM_athena__all_vali_vali.weights.h5"
#model = mprovider.from_weights(model_path)
model = mprovider.get_model()
print(model.summary())

2025-06-22 18:27:57.229918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38668 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500, 9)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 500, 128)     9344        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 500, 128)    512         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 500, 128)     0           ['batch_normalization[0][0]']

In [ ]:
# Load data
dprovider = SHLDataProviderRaw(datapath, "train", "all", silent=False)
x, y = dprovider.load_data()

dprovider_val = SHLDataProviderRaw(datapath, "validation", "all", silent=False)
x_2, y_2 = dprovider_val.load_data()

Loading labels from: /net/tscratch/people/plgjsikora/data/train/Hips/Label.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Acc_x.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Acc_y.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Acc_z.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Gyr_x.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Gyr_y.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Gyr_z.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Mag_x.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Mag_y.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Hips/Mag_z.txt
Loading labels from: /net/tscratch/people/plgjsikora/data/train/Torso/Label.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Torso/Acc_x.txt
Loading data from: /net/tscratch/people/plgjsikora/data/train/Torso/Ac

In [ ]:
print(x.shape, y.shape)
print(x_2.shape, y_2.shape)

In [ ]:
x_t = np.concatenate([x, x_2], axis=0)
y_t = np.concatenate([y, y_2], axis=0)

In [ ]:
print(x_t.shape, y_t.shape)

In [ ]:
# prepare data

x_t[np.isnan(x_t)] = 0




#x_n = x
#print(x.T.shape)
#for i in range(len(x.T)):
  #print(np.max(x.T[i]), np.min(x.T[i]))
  #x_n.T[i] = (x.T[i] - np.min(x.T[i])) / (np.max(x.T[i]) - np.min(x.T[i]))
  #print("-----------------------------")
#print(x_n.shape)
#print(np.min(x_n), np.max(x_n))

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train, x_val, y_train, y_val = train_test_split(x_t, y_t, test_size = 0.2, random_state=42)

In [ ]:
# compile model

from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

model_checkpoint_path =  f"/net/tscratch/people/plgjsikora/models/MLSTM_athena_final{today}.weights.h5"
model_checkpoint = ModelCheckpoint(model_checkpoint_path, verbose=1, mode='auto',
                                       monitor='val_accuracy', save_best_only=True, save_weights_only=True)

optm = Adam(learning_rate=5e-4)

callback_list = [model_checkpoint]
model.compile(optimizer=optm, loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
#history = model.fit(x_train, y_train, batch_size=256, epochs=50, verbose=1, shuffle=True, callbacks=callback_list, validation_freq=1, validation_data=(x_val, y_val))

In [ ]:
#import pickle
#with open(f'/net/tscratch/people/plgjsikora/histfinal{today}.pickle', 'wb') as file_pi:
 #   pickle.dump(history.history, file_pi)

In [ ]:
#import matplotlib.pyplot as plt
#disp.plot()
#plt.show()
#plt.savefig(f"/net/tscratch/people/plgjsikora/cm{today}_valitovali_50epochs.png")

In [ ]:
class ModelProviderNoAntiOverfitting:
  def __init__(self, seq_len = 500, nfeatures = 9, nclasses = 8):
    '''
    | Initializer model provider
    | Args:
    | seq_len - length of time series analysed
    | nfeatures - number of features of each time point
    | nclasses - number of target classes
    | Example usage:
    | provider = ModelProvider(500, 9, 8)
    | model = provider.get_model()
    '''
    self.seq_len = seq_len
    self.nfeatures = nfeatures
    self.nclasses = nclasses
    self.model = None


  def build_model(self):
    '''
    | Build model for SHL Challenge
    | Reimplementation of a model from https://github.com/titu1994/MLSTM-FCN for newer keras
    '''

    def squeeze_excite_block(input):
      ''' Create a squeeze-excite block
      Args:
          input: input tensor
          filters: number of output filters
          k: width factor

      Returns: a keras tensor
      '''
      filters = input.shape[-1] # channel_axis = -1 for TF

      se = GlobalAveragePooling1D()(input)
      se = Reshape((1, filters))(se)
      se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
      se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
      se = multiply([input, se])
      return se

    ip = Input(shape=(self.seq_len, self.nfeatures), dtype=float)
    x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = LSTM(8)(x)

    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform', data_format='channels_last')(ip)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform', data_format='channels_last')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform', data_format='channels_last')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(self.nclasses, activation='softmax')(x)
    self.model = Model(ip, out)

  def from_weights(self, weightpath):
    '''
    Get model from existing weights
    '''
    self.build_model()
    self.model.load_weights(weightpath)
    return self.model

  def get_model(self):
    if self.model is None:
      self.build_model()
    return self.model

In [ ]:
noantiover = ModelProviderNoAntiOverfitting(nfeatures=9)
nao_model = noantiover.get_model()
nao_model_checkpoint_path =  f"/net/tscratch/people/plgjsikora/models/MLSTM_athena_final_noantiover_{today}.weights.h5"
nao_model_checkpoint = ModelCheckpoint(nao_model_checkpoint_path, verbose=1, mode='auto',
                                       monitor='val_accuracy', save_best_only=True, save_weights_only=True)

optm = Adam(learning_rate=5e-4)

callback_list = [nao_model_checkpoint]
nao_model.compile(optimizer=optm, loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
#nao_history = nao_model.fit(x_train, y_train, batch_size=256, epochs=50, verbose=1, shuffle=True, callbacks=callback_list, validation_freq=1, validation_data=(x_val, y_val))

In [ ]:
#with open(f'/net/tscratch/people/plgjsikora/naohist150epochs_{today}.pickle', 'wb') as file_pi:
    #pickle.dump(nao_history.history, file_pi)

In [ ]:
#ao = ModelProvider(nfeatures = 9)
#ao_bestmodel = ao.from_weights("/net/tscratch/people/plgjsikora/models/MLSTM_athena_final2025-06-21.weights.h5")
#preds = ao_bestmodel.predict(x_t)
nao = ModelProviderNoAntiOverfitting(nfeatures = 9)
nao_bestmodel = nao.from_weights(f"/net/tscratch/people/plgjsikora/models/MLSTM_athena_final_noantiover_{today}.weights.h5")
naopreds = nao_bestmodel.predict(x_t)
#np.save(f"/net/tscratch/people/plgjsikora/aopreds_{today}.npy", preds)
np.save(f"/net/tscratch/people/plgjsikora/naopreds_{today}.npy", naopreds)